In [22]:
# -*- coding: utf-8 -*-
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.optimizers import SGD
import math

In [23]:
# 이미지 데이터 경로 설정
train_path = r'/content/drive/MyDrive/음식'
valid_path = r'/content/drive/MyDrive/음식'

test_path = r'/content/drive/MyDrive/음식'

In [24]:
# 이미지 크기 설정
img_width, img_height = 224, 224

# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.5,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.5,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# 배치 크기 조정
batch_size = 64

train_generator = train_datagen.flow_from_directory(train_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')

Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [25]:
# VGG16 모델 로딩 및 학습 레이어 제거
input_tensor = Input(shape=(img_width, img_height, 3))
vgg16_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
last_layer = vgg16_model.output
last_layer = Flatten()(last_layer)
last_layer = Dense(256, activation='relu')(last_layer)
last_layer = Dropout(0.5)(last_layer)
output_layer = Dense(train_generator.num_classes, activation='softmax')(last_layer)

model = Model(inputs=vgg16_model.input, outputs=output_layer)

In [26]:
# 미세 조정을 위한 학습 설정
for layer in model.layers[:10]:
    layer.trainable = False


# 학습률 조정
lr = 0.01
sgd = SGD(lr=lr, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [27]:
# 모델 학습 및 평가
steps_per_epoch = math.floor(train_generator.samples / train_generator.batch_size)
validation_steps = math.floor(valid_generator.samples / valid_generator.batch_size)
test_steps = math.ceil(test_generator.samples / test_generator.batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)

scores = model.evaluate_generator(test_generator, steps=test_steps)
print("Test accuracy: %.2f%%" % (scores[1]*100))

<ipython-input-27-19dca02107cf>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, validation_data=valid_generator, validation_steps=validation_steps)


Epoch 1/5
6/6 [==============================] - 433s 76s/step - loss: 5.9886 - accuracy: 0.2440 - val_loss: 1.4616 - val_accuracy: 0.2526
Epoch 2/5
6/6 [==============================] - 426s 75s/step - loss: 4.3380 - accuracy: 0.2679 - val_loss: 7.2405 - val_accuracy: 0.2448
Epoch 3/5
6/6 [==============================] - 497s 89s/step - loss: 8.5166 - accuracy: 0.2708 - val_loss: 3.1822 - val_accuracy: 0.2474
Epoch 4/5
6/6 [==============================] - 426s 75s/step - loss: 6.6134 - accuracy: 0.2679 - val_loss: 4866.4409 - val_accuracy: 0.2500
Epoch 5/5
6/6 [==============================] - 424s 75s/step - loss: nan - accuracy: 0.2946 - val_loss: nan - val_accuracy: 0.2526


<ipython-input-27-19dca02107cf>:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, steps=test_steps)


Test accuracy: 25.00%


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
